In [2]:
import pandas as pd          # Powerful data manipulation and analysis library
import yfinance as yf        # Used to fetch stock data from Yahoo Finance
import numpy as np           # Provides efficient numerical operations
import time
# For visualization or further analysis (optional for now)
import matplotlib.pyplot as plt  


In [3]:
import requests
from bs4 import BeautifulSoup

# Scrape the Wikipedia S&P 500 companies page
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)
response.raise_for_status()  # raises an error if the request fails

# Parse HTML and extract tickers
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {'id': 'constituents'})

tickers = []
for row in table.find_all('tr')[1:]:  # skip header row
    cells = row.find_all('td')
    if cells:
        ticker = cells[0].text.strip().replace('.', '-')  # Yahoo uses '-' instead of '.'
        tickers.append(ticker)

print("Total S&P 500 tickers found:", len(tickers))
print("Sample tickers:", tickers[:10])


Total S&P 500 tickers found: 503
Sample tickers: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


In [4]:
# Define financial metrics for extraction
metrics = [
    'marketCap',         # Market capitalization (numeric, will be formatted in B/T)
    'priceToBook',       # Price to book ratio
    'trailingPE',        # Trailing price to earnings ratio
    'forwardPE',         # Forward price to earnings ratio
    'dividendYield',     # Dividend yield
    'buybackYield',      # Buyback yield
    'debtToEquity',      # Debt to equity ratio
    'currentRatio',      # Current ratio (liquidity measure)
    'returnOnEquity',    # Return on equity (ROE)
    'roic',              # Return on invested capital (ROIC) - may be missing
    'grossMargins',      # Gross margin (percentage)
    'sector'             # Sector classification
]

# Collect metrics for each S&P 500 company
financials = {}
failures = []

for idx, ticker in enumerate(tickers):
    try:
        yf_ticker = yf.Ticker(ticker)
        info = yf_ticker.info
        row = {metric: info.get(metric, None) for metric in metrics}
        financials[ticker] = row
    except Exception as e:
        failures.append((ticker, str(e)))
    if idx % 50 == 0:
        print(f"Processed {idx+1}/{len(tickers)} tickers...")
    time.sleep(0.5)  # Add delay to reduce risk of connection issues with Yahoo Finance

# Convert collected data to DataFrame
df_financials = pd.DataFrame.from_dict(financials, orient='index')
df_financials.columns = [
    'Market Cap', 'P/B', 'Trailing P/E', 'Fwd P/E', 'Div Yield',
    'Buyback Yield', 'Debt/Equity', 'Current Ratio', 'ROE',
    'ROIC', 'Gross Margin', 'Sector'
]

# Format Market Cap: billions and trillions with appropriate separator
def format_market_cap(market_cap):
    if pd.isnull(market_cap):
        return None
    market_cap = float(market_cap)
    if market_cap >= 1e12:
        return f"{market_cap/1e9:,.2f}B".replace(",", "'")  # trillions as xxxx.yyB
    elif market_cap >= 1e9:
        return f"{market_cap/1e9:,.2f}B"
    elif market_cap >= 1e6:
        return f"{market_cap/1e6:,.2f}M"
    else:
        return f"{market_cap:.0f}"

# Format multiples: round to 2 decimals and add 'x' suffix
def format_multiple(val):
    try:
        return f"{float(val):.2f}x" if not pd.isnull(val) else None
    except:
        return None

# Format gross margin as percent, rounded to 2 decimals
def format_gross_margin(val):
    try:
        return f"{float(val)*100:.2f}%" if not pd.isnull(val) else None
    except:
        return None

# Apply formatting
df_financials['Market Cap'] = df_financials['Market Cap'].apply(format_market_cap)
df_financials['P/B'] = df_financials['P/B'].apply(format_multiple)
df_financials['Trailing P/E'] = df_financials['Trailing P/E'].apply(format_multiple)
df_financials['Fwd P/E'] = df_financials['Fwd P/E'].apply(format_multiple)
df_financials['Debt/Equity'] = df_financials['Debt/Equity'].apply(format_multiple)
df_financials['Current Ratio'] = df_financials['Current Ratio'].apply(format_multiple)
df_financials['ROE'] = df_financials['ROE'].apply(format_multiple)
df_financials['ROIC'] = df_financials['ROIC'].apply(format_multiple)
df_financials['Gross Margin'] = df_financials['Gross Margin'].apply(format_gross_margin)

# Sort DataFrame by Market Cap (descending)
df_financials['Market Cap Sort'] = df_financials['Market Cap'].replace(
    {"None": 0}).apply(lambda x: float(x.replace("'", "").replace("B", "")) if isinstance(x, str) and "B" in x else 0)
df_financials = df_financials.sort_values(by="Market Cap Sort", ascending=False)
df_financials = df_financials.drop(columns=['Market Cap Sort'])

# Display the top 10 largest companies
df_financials.head(10)


Processed 1/503 tickers...
Processed 51/503 tickers...
Processed 101/503 tickers...
Processed 151/503 tickers...
Processed 201/503 tickers...
Processed 251/503 tickers...
Processed 301/503 tickers...
Processed 351/503 tickers...
Processed 401/503 tickers...
Processed 451/503 tickers...
Processed 501/503 tickers...


,Market Cap,P/B,Trailing P/E,Fwd P/E,Div Yield,Buyback Yield,Debt/Equity,Current Ratio,ROE,ROIC,Gross Margin,Sector
NVDA,4'630.07B,46.24x,54.03x,46.16x,0.02,None,10.58x,4.21x,1.09x,None,69.85%,Technology
AAPL,4'033.21B,54.58x,36.52x,32.78x,0.38,None,152.41x,0.89x,1.71x,None,46.91%,Technology
MSFT,3'791.85B,10.45x,36.23x,34.13x,0.71,None,33.15x,1.40x,0.32x,None,68.76%,Technology
GOOG,3'364.48B,8.65x,27.32x,30.95x,0.30,None,11.42x,1.75x,0.35x,None,59.17%,Communication Services
GOOGL,3'364.48B,8.63x,27.31x,30.85x,0.30,None,11.42x,1.75x,0.35x,None,59.17%,Communication Services
AMZN,2'539.78B,6.79x,33.10x,38.16x,NaN,None,43.41x,1.01x,0.24x,None,50.05%,Consumer Cyclical
AVGO,1'617.22B,5.78x,87.81x,55.50x,0.69,None,166.03x,1.50x,0.27x,None,77.19%,Technology
META,1'537.24B,7.92x,26.96x,24.09x,0.34,None,26.31x,1.98x,0.33x,None,82.01%,Communication Services
TSLA,1'344.80B,16.81x,276.95x,124.80x,NaN,None,17.08x,2.07x,0.07x,None,17.01%,Consumer Cyclical
BRK-B,1'107.22B,0.00x,16.29x,25.35x,NaN,None,18.17x,2.72x,0.10x,None,24.36%,Financial Services
